In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# ===== Cấu hình Chrome =====
chrome_options = Options()
chrome_options.add_argument("--headless")  # bỏ nếu muốn xem trình duyệt
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# ===== Danh sách URL muốn crawl =====
urls = {
    "Điện thoại": "https://www.lazada.vn/catalog/?q=điện%20thoại",
    "Quần áo": "https://www.lazada.vn/catalog/?q=quần%20áo", 
    "Tủ lạnh": "https://www.lazada.vn/catalog/?q=tủ%20lạnh",
    "Tivi": "https://www.lazada.vn/catalog/?q=ti%20vi"
}

# ===== Hàm scroll để load thêm sản phẩm =====
def scroll_and_count():
    """Scroll và đếm sản phẩm có giá"""
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    driver.execute_script("window.scrollBy(0, -200);")
    time.sleep(2)
    
    # Đếm sản phẩm có giá hợp lệ
    products = driver.find_elements(By.CSS_SELECTOR, "div.RfADt")
    valid_count = 0
    
    for product in products:
        try:
            # Kiểm tra nhanh xem có giá không
            full_text = product.text
            if "₫" in full_text and any(char.isdigit() for char in full_text):
                valid_count += 1
        except:
            continue
    
    return len(products), valid_count

# ===== Hàm crawl dữ liệu 1 URL với đảm bảo tối thiểu =====
def crawl_lazada(url, category, min_items=80):
    print(f"🔍 Đang crawl {category} (cần tối thiểu {min_items} sản phẩm có giá)...")
    driver.get(url)
    time.sleep(5)
    
    data = []
    scroll_attempts = 0
    max_attempts = 8  # Tăng số lần thử
    last_valid_count = 0
    no_progress_count = 0
    
    while len(data) < min_items and scroll_attempts < max_attempts:
        scroll_attempts += 1
        print(f"  🔄 Lần scroll thứ {scroll_attempts}...")
        
        # Scroll và đếm
        total_products, estimated_valid = scroll_and_count()
        print(f"  📊 Tổng sản phẩm: {total_products}, ước tính có giá: {estimated_valid}")
        
        # Kiểm tra tiến trình
        if estimated_valid <= last_valid_count:
            no_progress_count += 1
            if no_progress_count >= 3:
                print(f"  ⚠️  Không có tiến trình trong 3 lần scroll, dừng lại")
                break
        else:
            no_progress_count = 0
        
        last_valid_count = estimated_valid
        
        # ===== Thay thế logic xử lý products bằng processed_index =====
        processed_index = 0
        products = driver.find_elements(By.CSS_SELECTOR, "div.RfADt")
        while len(data) < min_items:
            # nếu đã duyệt hết products thì scroll thêm và cập nhật lại
            if processed_index >= len(products):
                # scroll thêm 1 lần để load product mới
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                products = driver.find_elements(By.CSS_SELECTOR, "div.RfADt")
                # nếu không có thêm phần tử thì dừng vòng lặp
                if processed_index >= len(products):
                    print("  ⚠️ Không load thêm product mới, dừng việc lấy thêm.")
                    break

            # lấy product hiện tại và tăng processed_index (đảm bảo không lặp)
            product = products[processed_index]
            processed_index += 1

            try:
                # đảm bảo visible để lazy-load giá
                try:
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", product)
                except:
                    pass
                time.sleep(0.3)

                # Lấy tên
                try:
                    name_element = product.find_element(By.CSS_SELECTOR, "a")
                    name = name_element.get_attribute("title") or name_element.text.strip()
                    if not name:
                        name = product.text.strip().split('\n')[0][:100]
                except:
                    name = "N/A"

                # Lấy giá (giữ nguyên các phương pháp của bạn)
                price = "N/A"
                price_selectors = [
                    ".//following::span[@class='ooOxS'][1]",
                    ".//following::span[contains(@class, 'currency')]",
                    ".//following::span[contains(text(), '₫')]",
                    ".//span[contains(@class, 'price')]",
                    ".//span[contains(text(), '₫')]",
                    ".//following-sibling::*//span[contains(text(), '₫')]",
                    ".//ancestor::*//span[contains(text(), '₫')]"
                ]
                for selector in price_selectors:
                    try:
                        price_element = product.find_element(By.XPATH, selector)
                        price_text = price_element.get_attribute("textContent") or price_element.text
                        price_text = price_text.strip()
                        if price_text and "₫" in price_text and any(c.isdigit() for c in price_text):
                            price = price_text
                            break
                    except:
                        continue

                # fallback: tìm trong text node nếu selector không có
                if price == "N/A":
                    try:
                        full_text = product.get_attribute("textContent") or product.text or ""
                        import re
                        m = re.search(r'[\d\.,\s]+₫', full_text)
                        if m:
                            price = m.group().strip()
                    except:
                        pass

                # sold count (giữ nguyên)
                try:
                    sold_selectors = [
                        ".//following::span[contains(text(),'Đã bán')]",
                        ".//span[contains(text(),'Đã bán')]",
                        ".//following::*[contains(text(),'Đã bán')]"
                    ]
                    sold_count = "N/A"
                    for selector in sold_selectors:
                        try:
                            sold_element = product.find_element(By.XPATH, selector)
                            sold_count = sold_element.get_attribute("textContent") or sold_element.text
                            sold_count = sold_count.strip()
                            break
                        except:
                            continue
                except:
                    sold_count = "N/A"

                # Thêm nếu có giá
                if price != "N/A" and price.strip() and "₫" in price:
                    data.append({
                        "Category": category,
                        "Name": name,
                        "Price": price,
                        "SoldCount": sold_count
                    })
                    print(f"  ✅ SP {len(data)}: {name[:40]}... - {price}")

                else:
                    print(f"  ⚠️ Bỏ qua product idx {processed_index}: không có giá hợp lệ")

            except Exception as e:
                print(f"  ❌ Lỗi khi xử lý product idx {processed_index}: {e}")
                continue
        
        print(f"  📈 Hiện có {len(data)}/{min_items} sản phẩm hợp lệ")
        
        # Nếu đã đủ, thoát
        if len(data) >= min_items:
            print(f"  🎉 Đã đủ {min_items} sản phẩm!")
            break
            
        time.sleep(3)
    
    final_count = len(data)
    if final_count >= min_items:
        print(f"  ✅ [{category}] Thành công: {final_count} sản phẩm")
    else:
        print(f"  ⚠️  [{category}] Chỉ lấy được {final_count}/{min_items} sản phẩm")
    
    return data

# ===== Crawl toàn bộ URL =====
all_data = []
min_products_per_category = 80

for category, url in urls.items():
    results = crawl_lazada(url, category, min_items=min_products_per_category)
    all_data.extend(results)
    print(f"🔄 Nghỉ 5 giây trước khi crawl category tiếp theo...\n")
    time.sleep(5)

# ===== Lưu ra Excel =====
if all_data:
    df = pd.DataFrame(all_data)
    df.to_excel("lazada_products_minimum_80.xlsx", index=False)
    print(f"✅ Crawl xong! Tổng cộng {len(all_data)} sản phẩm được lưu vào lazada_products_minimum_80.xlsx")
    
    # Thống kê chi tiết
    print("\n📈 Thống kê theo danh mục:")
    category_stats = df.groupby('Category').size()
    total_categories = len(category_stats)
    success_categories = sum(1 for count in category_stats.values() if count >= min_products_per_category)
    
    for cat, count in category_stats.items():
        status = "✅" if count >= min_products_per_category else "⚠️ "
        print(f"  {status} {cat}: {count} sản phẩm")
    
    print(f"\n📊 Tổng kết: {success_categories}/{total_categories} danh mục đạt tối thiểu {min_products_per_category} sản phẩm")
    
    # Kiểm tra giá hợp lệ
    valid_prices = df[df['Price'] != 'N/A']
    print(f"💰 Sản phẩm có giá hợp lệ: {len(valid_prices)}/{len(df)} ({len(valid_prices)/len(df)*100:.1f}%)")
    
else:
    print("❌ Không thu thập được dữ liệu nào!")

driver.quit()

🔍 Đang crawl Điện thoại (cần tối thiểu 80 sản phẩm có giá)...
  🔄 Lần scroll thứ 1...
  📊 Tổng sản phẩm: 40, ước tính có giá: 0
  ✅ SP 1: Điện thoại realme C61 RAM 6GB ROM 128GB ... - 719.000 ₫
  ✅ SP 2: Điện thoại TECNO SPARK 40 (8GB/256GB) - ... - 4.140.000 ₫
  ✅ SP 3: Điện thoại TECNO SPARK 40 (6GB/128GB) - ... - 3.440.000 ₫
  ✅ SP 4: Điện thoại thông minh Xia0mi Redmi A5- 4... - 679.000 ₫
  ✅ SP 5: Galax A55 điện thoại 16GB RAM +1TB ROM M... - 300.000 ₫
  ✅ SP 6: Điện thoại OPP0 A5i Pr0 8GB/128GB | Thiế... - 725.000 ₫
  ✅ SP 7: Điện Thoại Kyocera Torque G05 - 5G TD-LT... - 2.190.000 ₫
  ✅ SP 8: Điện thoại Xiaomi Mi 11 Lite 5G Ram 8GB ... - 278.000 ₫
  ✅ SP 9: Galax S24 Ultra điện thoại gốc mớii 16GB... - 300.000 ₫
  ✅ SP 10: Điện thoại A.Oppo A97 5G 8GB/256GB Pin 5... - 1.975.000 ₫
  ✅ SP 11: Galax S24 ultra điện thoại 16GB RAM +1TB... - 300.000 ₫
  ✅ SP 12: Điện thoại realme 13+ 5G RAM 8GB ROM 256... - 955.000 ₫
  ✅ SP 13: Galax S24 điện thoại gốc mớii 16GB RAM +... - 1.199.000 ₫


TypeError: 'numpy.ndarray' object is not callable